HATE DETECTION : H2H

In [56]:
import pandas as pd
df = pd.read_csv("random_train.csv")

In [57]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,target
0,20d780bf2acc0bb9,In keeping with the Northern Ireland wikipedia...,0,0,0,0,0,0,0
1,0558f1dd8c98988d,The answer to that question has finally been f...,0,0,0,0,0,0,0
2,5eb71f3a0d810d2c,. UR SUCH A FUCKING GEEK ASS MOFO! UR MOTHERS ...,1,1,1,0,1,0,1
3,ccf66b8308935bfe,"OK, I'll have a look over it and let you know ...",0,0,0,0,0,0,0
4,6319df121dd73181,{unblock|Unblock me at once!}},0,0,0,0,0,0,0


In [58]:
df.drop(columns = ['target'], inplace = True)

In [59]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,20d780bf2acc0bb9,In keeping with the Northern Ireland wikipedia...,0,0,0,0,0,0
1,0558f1dd8c98988d,The answer to that question has finally been f...,0,0,0,0,0,0
2,5eb71f3a0d810d2c,. UR SUCH A FUCKING GEEK ASS MOFO! UR MOTHERS ...,1,1,1,0,1,0
3,ccf66b8308935bfe,"OK, I'll have a look over it and let you know ...",0,0,0,0,0,0
4,6319df121dd73181,{unblock|Unblock me at once!}},0,0,0,0,0,0


In [60]:
# !pip install torch
import torch
import numpy as np
Y = []
for i in df.values:
    Y.append(torch.tensor(np.array(i[2:],dtype = np.float32)))
Y = torch.stack(Y) 
print(Y)
Y.shape

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0.],
        ...,
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])


torch.Size([25000, 6])

EMBEDDINGS OF TWEETS

In [61]:
import pickle
import pandas as pd
import json
from tqdm import tqdm
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer 
model_sen = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


In [62]:
# X = []
# for i in tqdm(df["comment_text"]):
#     j = torch.tensor(model_sen.encode(i))
#     X.append(j) 

In [63]:
# X = torch.stack(X) 

In [64]:
# #saving the embeddings for future use
# with open('X.pkl', 'wb') as f:
#     pickle.dump(X, f) 

# print("Stacked tensor saved to stacked_tensor.pkl")

In [65]:
import torch
import pickle

# Load the embeddings from the pickle file
with open('X.pkl', 'rb') as f:
    X = pickle.load(f)
X = X.unsqueeze(1)
# Print the loaded tensor
print(X)


tensor([[[-0.0064,  0.0095, -0.0159,  ..., -0.0303,  0.0116,  0.0360]],

        [[ 0.0223, -0.0205,  0.0037,  ..., -0.0511, -0.0188,  0.0055]],

        [[-0.0434,  0.0195, -0.0081,  ...,  0.0123, -0.0408, -0.0310]],

        ...,

        [[-0.0443,  0.0508, -0.0243,  ...,  0.0097,  0.0301, -0.0234]],

        [[-0.0204,  0.0148,  0.0024,  ..., -0.0210, -0.0555,  0.0149]],

        [[ 0.0763,  0.0464,  0.0062,  ..., -0.0104, -0.0767,  0.0558]]])


In [66]:
X.shape

torch.Size([25000, 1, 768])

In [67]:
# combing X and Y to form a dataset
# !pip install tensorflow
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(5)

In [68]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [69]:
train_generator = train.as_numpy_iterator()
train_generator.next()

2024-11-12 13:28:18.777527: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(array([[[ 0.08325461,  0.00196811,  0.0030059 , ...,  0.01355089,
          -0.01876356, -0.01708327]],
 
        [[ 0.02172103,  0.06832755, -0.00109624, ...,  0.05685737,
          -0.05219594, -0.01677671]],
 
        [[-0.02178163,  0.10040811,  0.01068756, ...,  0.00071513,
          -0.01322643,  0.01048273]],
 
        ...,
 
        [[-0.00512984,  0.03185592,  0.0106814 , ..., -0.00739371,
           0.03378417, -0.01757451]],
 
        [[ 0.05113736, -0.00228241,  0.01081558, ...,  0.05385644,
          -0.0387287 , -0.01703592]],
 
        [[ 0.09448456, -0.00915827, -0.01450976, ...,  0.05159661,
          -0.0403772 ,  0.04093753]]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0

MODEL

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Activation
import torch
# Define the Keras model
model = Sequential([
    Bidirectional(LSTM(768, activation = 'tanh')),
    Dense(128),
    Activation('relu'),
    Dense(16),
    Activation('tanh'),
    Dense(6),
    Activation('sigmoid')
])
# model1(X_text_train.unsqueeze(1))

# Compile the model
# optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer= 'Adam')

# Print the model summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [71]:
history = model.fit(train, epochs = 2, validation_data= val)

Epoch 1/2
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1514

2024-11-12 13:29:22.472221: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1094/1094 ━━━━━━━━━━━━━━━━━━━━ 44s 38ms/step - loss: 0.1514 - val_loss: 0.0715
Epoch 2/2
   3/1094 ━━━━━━━━━━━━━━━━━━━━ 45s 42ms/step - loss: 0.0707 

2024-11-12 13:29:24.308946: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1093/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0647

2024-11-12 13:30:07.108112: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1094/1094 ━━━━━━━━━━━━━━━━━━━━ 44s 40ms/step - loss: 0.0647 - val_loss: 0.0648


2024-11-12 13:30:08.605501: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [72]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 1536)           │     9,443,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,926,692 (110.35 MB)

 Trainable params: 9,642,230 (36.78 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,284,462 (73.56 MB)

PREDICTIONS

In [73]:
X_test = torch.tensor(model_sen.encode("I'll kill you")).unsqueeze(0).unsqueeze(0)
X_test

tensor([[[ 7.1044e-02,  4.8869e-02, -5.5874e-03,  4.4661e-03, -6.5701e-03,
           3.8937e-02, -3.0229e-02,  4.2765e-02, -1.2309e-02,  1.3392e-02,
           2.7514e-02,  2.2837e-02,  7.3720e-02,  5.7937e-02, -6.5132e-03,
           6.2169e-03,  1.0437e-02, -5.4102e-03,  5.1908e-02, -2.3087e-02,
          -5.3403e-03,  2.7886e-02, -6.2808e-03,  1.0456e-02,  1.0376e-02,
           1.3865e-02,  5.3587e-02,  2.5933e-02, -3.4685e-03, -3.3729e-02,
          -9.2243e-03, -4.4367e-02, -4.2160e-02,  2.2287e-02,  1.7971e-06,
          -4.2737e-02,  6.8327e-02, -1.1885e-03, -5.4989e-02,  1.0131e-01,
           4.8786e-03, -7.5937e-02, -2.4148e-02, -3.0298e-02,  1.2081e-02,
          -2.1161e-02,  9.9956e-03,  2.6894e-02,  2.1409e-02, -5.1983e-03,
          -8.0993e-03, -5.3718e-02,  2.8362e-02, -2.2426e-02,  7.7232e-03,
           1.9633e-02,  1.5691e-02,  3.1930e-02, -1.3382e-02, -3.5782e-03,
          -2.1913e-02, -7.3919e-03, -1.0127e-02,  4.6985e-02,  8.1480e-03,
           1.1517e-03,  5

In [74]:
Y_test = model.predict(X_test)
Y_test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step


array([[0.6332574 , 0.01883683, 0.15989828, 0.05047224, 0.14624305,
        0.11661158]], dtype=float32)

In [75]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [76]:
test_batch_X, test_batch_Y = test.as_numpy_iterator().next()

2024-11-12 13:30:33.075489: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [104]:
predict_Y = (model.predict(test_batch_X) > 0.20).astype(int)
predict_Y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [105]:
test_batch_Y

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.]], dtype=float32)

EVALUATING THE MODEL

In [106]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(test_batch_Y, predict_Y, average='micro')
recall = recall_score(test_batch_Y, predict_Y, average= 'micro')
accuracy = (predict_Y == test_batch_Y).astype(float).mean().item()
print("accuracy: ",accuracy) 
print("recall: ",recall)
print("precision: ",precision)

accuracy:  0.9583333333333334
recall:  0.5714285714285714
precision:  0.8


SAVE MODEL

In [63]:
from tensorflow.keras.models import save_model

save_model(model, 'hate_detection.keras')


In [39]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('hate_detection.keras') 

# Now you can use the loaded_model for predictions or further training


In [40]:
def evaluate_hate(tweet):
    eval_X = torch.tensor(model_sen.encode(tweet)).unsqueeze(0).unsqueeze(0)
    res = model.predict(eval_X)
    result = f'tweet: "{tweet}"\n'
    for i, col in enumerate(df.columns[2:]):
        result += f'{col}: {res[0][i]>0.5}\n'
    return result

TEST HERE

In [112]:
print("WELCOME TO HATE DETECTOR: H2H")
tweet = input("Enter the tweet to check")
print(evaluate_hate(tweet)) 

WELCOME TO HATE DETECTOR: H2H
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
tweet: "You are a very dumb guy"
toxic: True
severe_toxic: False
obscene: False
threat: False
insult: False
identity_hate: False

